<h1> Code for Lychee Yield Prediction </h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# multivariate linear regression with regularization
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# support vector machine regression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
# neural network
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional, BatchNormalization
# normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
# score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV
# import keras
import tensorflow.keras
from tensorflow.keras import optimizers, regularizers
# import regularizer
from tensorflow.keras.regularizers import l1, l2
# import matplotlib
import matplotlib.pyplot as plt
# os
import os

import pickle
from calendar import monthrange

In [2]:
def clean_text_to_number(df):
    '''
    convert all text to 0
    '''
    cols = df.columns
    type_list = []
    for col in cols:
        print(col)
        try:
            df[col].astype(float)
        except:
            for i in range(df[col].shape[0]):
                if isinstance(df[col].iloc[i], str):
                    df[col].iloc[i] = 0
    return df

In [3]:
# import data frame
rain_df     = pd.read_excel('rain_amount_2003-2019.xlsx')
humid_df    = pd.read_excel('relative_humid_2003-2019.xlsx')
temp_df     = pd.read_excel('temp_2003-2019.xlsx')
area_df     = pd.read_excel('area_2003-2019.xls', sheet_name = 'Sheet1')
lychee_yield_df = pd.read_excel('lycheeproduct.xlsx')

# extract data
rain_df     = rain_df.iloc[5:-5, :]
humid_df    = humid_df.iloc[5:-5, :]
temp_df     = temp_df.iloc[5:-5, :]

# reset index
rain_df     = rain_df.reset_index().drop(columns=['index'])
humid_df    = humid_df.reset_index().drop(columns=['index'])
temp_df     = temp_df.reset_index().drop(columns=['index'])

# set column name
rain_df.columns     = ['days', 'location', 'date', '1', '4', '7', '10', '13', '16', '19', '22', 'total']
humid_df.columns    = ['days', 'location', 'date', '1', '4', '7', '10', '13', '16', '19', '22', 'mean']
temp_df.columns     = ['days', 'location', 'date', '1', '4', '7', '10', '13', '16', '19', '22', 'mean']
lychee_yield_df.columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'total', 'year']
area_df.columns     = ['year', 'district', 'code', 'province', 'allarea', 'yieldarea', 'yield', 'yieldperarea']
area_df['year']     = area_df['year'] - 543

# ดึงค่า date เก็บไว้ก่อน
all_datetime    = pd.to_datetime(rain_df['date'])

rain_df     = rain_df.drop(columns = ['location', 'days', 'date'])
humid_df    = humid_df.drop(columns =['location', 'days', 'date'])
temp_df     = temp_df.drop(columns =['location', 'days', 'date'])
area_df     = area_df.drop(columns = ['province'])

print('rain amount dataframe')
print(rain_df.shape)
print('humid dataframe')
print(humid_df.shape)
print('temperature dataframe')
print(temp_df.shape)
print('rain dataframe sample')
print(rain_df.head(5))
print('area dataframe sample')
print(area_df.head(5))

rain amount dataframe
(6083, 9)
humid dataframe
(6083, 9)
temperature dataframe
(6083, 9)
rain dataframe sample
     1    4    7   10 13    16    19   22 total
0    0    0    0    0  0     0     0  1.2   1.2
1  4.4  0.2    5  2.6  0  17.4     0    0  29.6
2    0    0    0    0  0     0     0    0     -
3    0    0    0    0  0     0     0    0     -
4    0  0.6  0.7  2.5  2   7.3  15.1  1.4  29.6
area dataframe sample
   year  district  code  allarea  yieldarea      yield  yieldperarea
0  1994         1    10    24562      16404   8366.040           510
1  1995         1    10    22053      15021   5798.106           386
2  1996         1    10    27955      18807   8068.203           429
3  1997         1    10    31203      23278  11429.498           491
4  1998         1    10    34200      23519   1481.697            63


In [4]:
# # get datetime from 3 hour data set
# all_year        = pd.DataFrame(all_datetime.dt.year)
# all_month       = pd.DataFrame(all_datetime.dt.month)
# all_day         = pd.DataFrame(all_datetime.dt.day)

# all_year.columns    = ['year']
# all_month.columns   = ['month']
# all_day.columns     = ['day']
# # concat to existing dataframe
# rain_df = pd.concat([all_year, all_month, all_day, rain_df], axis = 1)
# humid_df = pd.concat([all_year, all_month, all_day, humid_df], axis = 1)
# temp_df = pd.concat([all_year, all_month, all_day, temp_df], axis = 1)

# # clean text element and save in pickle form

# rain_df_clean = clean_text_to_number(rain_df)
# with open('rain_df_clean.pickle', 'wb') as f:
#     pickle.dump(rain_df_clean, f)
# humid_df_clean = clean_text_to_number(humid_df)
# with open('humid_df_clean.pickle', 'wb') as f:
#     pickle.dump(humid_df_clean, f)
# temp_df_clean = clean_text_to_number(temp_df)
# with open('temp_df_clean.pickle', 'wb') as f:
#     pickle.dump(temp_df_clean, f)

<h1> Load Data from pickle </h1> since clean dataframe take very long time. So after we clean it, we save dataframe in pickle form.

In [5]:
with open('rain_df_clean.pickle', 'rb') as f:
    rain_df_clean = pickle.load(f)
with open('humid_df_clean.pickle', 'rb') as f:
    humid_df_clean = pickle.load(f)
with open('temp_df_clean.pickle', 'rb') as f:
    temp_df_clean = pickle.load(f)

In [6]:
print(' --- rain amount dataframe sample ---')
print(rain_df_clean.head(5))
print(' --- humid dataframe sample ---')
print(humid_df_clean.head(5))
print(' --- temperature dataframe sample ---')
print(temp_df_clean.head(5))
print(' --- area dataframe sample ---')
print(area_df.head(5))
print(' --- lychee yield dataframe sample ---')
print(lychee_yield_df.head(5))

--- rain amount dataframe sample ---
   year  month  day    1    4    7   10 13    16    19   22 total
0  2003      1    1    0    0    0    0  0     0     0  1.2   1.2
1  2003      1    2  4.4  0.2    5  2.6  0  17.4     0    0  29.6
2  2003      1    3    0    0    0    0  0     0     0    0     0
3  2003      1    4    0    0    0    0  0     0     0    0     0
4  2003      1    5    0  0.6  0.7  2.5  2   7.3  15.1  1.4  29.6
 --- humid dataframe sample ---
   year  month  day   1   4   7  10  13  16  19  22 total
0  2003      1    1  93  95  95  87  75  73  89  93    88
1  2003      1    2  94  95  94  94  87  91  94  95    93
2  2003      1    3  95  95  95  87  70  60  92  94    86
3  2003      1    4  96  95  94  81  70  62  81  85    83
4  2003      1    5  88  93  93  91  85  95  95  95    92
 --- temperature dataframe sample ---
   year  month  day     1     4     7    10    13    16    19    22 total
0  2003      1    1  19.2  18.5  18.2    21  23.5  24.3  22.1  21.3    21
1

In [7]:
def groupby_col(self, col):
    '''
    return 
        1 keys 
        2 dictionary of sub_df ที่ keys คือแต่ละ element ใน col name
    '''

    output_dic = {}
    all_ele = sorted(list(set(self[col])))

    for ele in all_ele:
        sub_df = self.loc[self[col] == ele, :]
        output_dic[ele] = sub_df

    return all_ele, output_dic

def groupby_mean(self, col):
    '''
    return 
        1 keys 
        2 dictionary of sub_df ที่ keys คือแต่ละ element ใน col name
    '''

    output_dic = {}
    all_ele = sorted(list(set(self[col])))

    for ele in all_ele:
        sub_df = self.loc[self[col] == ele, :].mean(axis=0)
        output_dic[ele] = sub_df
    
    return all_ele, output_dic

def groupby_max(self, col):
    '''
    return 
        1 keys 
        2 dictionary of sub_df ที่ keys คือแต่ละ element ใน col name
    '''

    output_dic = {}
    all_ele = sorted(list(set(self[col])))

    for ele in all_ele:
        sub_df = self.loc[self[col] == ele, :].max(axis = 0)
        output_dic[ele] = sub_df
    
    return all_ele, output_dic

def groupby_min(self, col):
    '''
    return 
        1 keys 
        2 dictionary of sub_df ที่ keys คือแต่ละ element ใน col name
    '''

    output_dic = {}
    all_ele = sorted(list(set(self[col])))

    for ele in all_ele:
        sub_df = self.loc[self[col] == ele, :].min(axis = 0)
        output_dic[ele] = sub_df
    
    return all_ele, output_dic

def groupby_sum(self, col):
    '''
    return 
        1 keys 
        2 dictionary of sub_df ที่ keys คือแต่ละ element ใน col name
    '''

    output_dic = {}
    all_ele = sorted(list(set(self[col])))

    for ele in all_ele:
        sub_df = self.loc[self[col] == ele, :].sum(axis = 0)
        output_dic[ele] = sub_df
    
    return all_ele, output_dic

# set method to class
setattr(pd.core.frame.DataFrame, 'groupby_col', groupby_col)
setattr(pd.core.frame.DataFrame, 'groupby_mean', groupby_mean)
setattr(pd.core.frame.DataFrame, 'groupby_sum', groupby_sum)
setattr(pd.core.frame.DataFrame, 'groupby_max', groupby_max)
setattr(pd.core.frame.DataFrame, 'groupby_min', groupby_min)

In [8]:
def export_output(filename, year_train, month_train, y_train_true, y_train_pred, year_test, month_test, y_test_true, y_test_pred):
    
    dic_train = {
        'year':year_train.reshape(-1),
        'month':month_train.reshape(-1),
        'y_true': y_train_true.reshape(-1),
        'y_pred': y_train_pred.reshape(-1)
    }

    dic_test = {
        'year':year_test.reshape(-1),
        'month':month_test.reshape(-1),
        'y_true': y_test_true.reshape(-1),
        'y_pred': y_test_pred.reshape(-1)
    }
    
    df_train = pd.DataFrame(dic_train)
    df_test = pd.DataFrame(dic_test)
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    # Write centroids sheet
    df_train.to_excel(writer, sheet_name='train set')
    df_test.to_excel(writer, sheet_name='test set')
    # save output
    writer.save()


<h1> Rearrange for monthly data </h1>

In [9]:
## montly Temp 2004 - 2019
monthly_temp = np.array([1,1,1]).reshape(1,3)

all_year, sub_year_dic = temp_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and mean temp of each month
                daily_temp = np.array([year, month, sub_month_dic[month].iloc[-1]]).reshape(1,3)
                monthly_temp = np.append(monthly_temp, daily_temp, axis = 0)

monthly_temp = np.delete(monthly_temp, 0, axis = 0)
print(monthly_temp.shape)

(200, 3)


In [10]:
## monthly Humid 2004 - 2019
monthly_humid = np.array([1,1,1]).reshape(1,3)

all_year, sub_year_dic = humid_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and mean temp of each month
                daily_humid = np.array([year, month, sub_month_dic[month].iloc[-1]]).reshape(1,3)
                monthly_humid = np.append(monthly_humid, daily_humid, axis = 0)

monthly_humid = np.delete(monthly_humid, 0, axis = 0)
print(monthly_humid.shape)

(200, 3)


In [11]:
## monthly Rain 2004 - 2019
monthly_rain = np.array([1,1,1]).reshape(1,3)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and mean temp of each month
                daily_rain = np.array([year, month, sub_month_dic[month].iloc[-1]]).reshape(1,3)
                monthly_rain = np.append(monthly_rain, daily_rain, axis = 0)

monthly_rain = np.delete(monthly_rain, 0, axis = 0)
print(monthly_rain.shape)

(200, 3)


In [12]:
## monthly Lychee yield 2004 - 2018
monthly_lychee = np.array([1,1,1]).reshape(1,3)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, _ = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and lychee yield of each month
                monthly_lychee_temp = lychee_yield_df.loc[lychee_yield_df['year']==year,:]
                if monthly_lychee_temp.size != 0:
                    monthly_yield = np.array([year, month, monthly_lychee_temp.iloc[0, month-1]]).reshape(1,3)
                    monthly_lychee = np.append(monthly_lychee, monthly_yield, axis = 0)

monthly_lychee = np.delete(monthly_lychee, 0, axis = 0)
print(monthly_lychee.shape)

(180, 3)


In [13]:
## monthly Lychee yield 2004 - 2018 but use yearly yield
monthly_lychee_yearlyyield = np.array([1,1,1]).reshape(1,3)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, _ = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and lychee yield of each month
                monthly_lychee_temp = lychee_yield_df.loc[lychee_yield_df['year']==year,:]
                if monthly_lychee_temp.size != 0:
                    monthly_yield = np.array([year, month, monthly_lychee_temp.iloc[0, -2]]).reshape(1,3)
                    monthly_lychee_yearlyyield = np.append(monthly_lychee_yearlyyield, monthly_yield, axis = 0)

monthly_lychee_yearlyyield = np.delete(monthly_lychee_yearlyyield, 0, axis = 0)
print(monthly_lychee_yearlyyield[:5, :])

[[ 2004     1 18399]
 [ 2004     2 18399]
 [ 2004     3 18399]
 [ 2004     4 18399]
 [ 2004     5 18399]]


In [14]:
## monthly yield per area 1994 - 2018
monthly_area = np.array([1,1,1]).reshape(1,3)

all_year, _ = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, _ = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and lychee yield of each month
                monthly_area_temp = area_df.loc[area_df['year']==year,:]
                if monthly_area_temp.size != 0:
                    monthly_area_temp = np.array([year, month, monthly_area_temp.iloc[0, -1]]).reshape(1,3)
                    monthly_area = np.append(monthly_area, monthly_area_temp, axis = 0)

monthly_area = np.delete(monthly_area, 0, axis = 0)
print(monthly_area.shape)

(192, 3)


In [15]:
## monthly all yield area 1994 - 2018
monthly_yieldarea = np.array([1,1,1]).reshape(1,3)

all_year, _ = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, _ = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and lychee yield of each month
                monthly_area_temp = area_df.loc[area_df['year']==year,:]
                if monthly_area_temp.size != 0:
                    monthly_area_temp = np.array([year, month, monthly_area_temp.iloc[0, 4]]).reshape(1,3)
                    monthly_yieldarea = np.append(monthly_yieldarea, monthly_area_temp, axis = 0)

monthly_yieldarea = np.delete(monthly_yieldarea, 0, axis = 0)
print(monthly_area.shape)

(192, 3)


In [16]:
## month range 2004 - 2019
monthly_numday = np.array([1,1,1]).reshape(1,3)

all_year, _ = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, _ = sub_year_dic[year].groupby_mean('month')

        for month in all_month:
                # record year month and lychee yield of each month
                monthly_numday_temp = np.array([year, month, monthrange(year, month)[1]]).reshape(1,3)
                monthly_numday = np.append(monthly_numday,monthly_numday_temp, axis = 0)

monthly_numday = np.delete(monthly_numday, 0, axis = 0)
print(monthly_numday.shape)

(200, 3)


<h1> Rearrange for daily data </h1>

In [17]:
## daily Temp 2004 - 2019
daily_temp = np.array([1,1,1,1]).reshape(1,4)

# get yearly dic
all_year, sub_year_dic = temp_df_clean.groupby_col('year')

for year in all_year:
        # get monthly dic
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')

        for month in all_month:
            # get daily dic
            all_day, sub_day_dic = sub_month_dic[month].groupby_mean('day')       

            for day in all_day:
                # get daily dataframe
                sub_day_df = sub_day_dic[day]
                daily_tempo = np.array([year, month, day, sub_day_df.iloc[-1]]).reshape(1,4)
                daily_temp = np.append(daily_temp, daily_tempo, axis = 0)

daily_temp = np.delete(daily_temp, 0, axis = 0)
print(daily_temp[:6, :])

[[2.003e+03 1.000e+00 1.000e+00 2.100e+01]
 [2.003e+03 1.000e+00 2.000e+00 2.150e+01]
 [2.003e+03 1.000e+00 3.000e+00 2.180e+01]
 [2.003e+03 1.000e+00 4.000e+00 2.210e+01]
 [2.003e+03 1.000e+00 5.000e+00 2.020e+01]
 [2.003e+03 1.000e+00 6.000e+00 2.130e+01]]


In [18]:
## daily humid 2004 - 2019
daily_humid = np.array([1,1,1,1]).reshape(1,4)

# get yearly dic
all_year, sub_year_dic = humid_df_clean.groupby_col('year')

for year in all_year:
        # get monthly dic
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')

        for month in all_month:
            # get daily dic
            all_day, sub_day_dic = sub_month_dic[month].groupby_mean('day')       

            for day in all_day:
                # get daily dataframe
                sub_day_df = sub_day_dic[day]
                daily_tempo = np.array([year, month, day, sub_day_df.iloc[-1]]).reshape(1,4)
                daily_humid = np.append(daily_humid, daily_tempo, axis = 0)

daily_humid = np.delete(daily_humid, 0, axis = 0)
print(daily_humid.shape)

(6083, 4)


In [19]:
## daily rain 2004 - 2019
daily_rain = np.array([1,1,1,1]).reshape(1,4)

# get yearly dic
all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        # get monthly dic
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')

        for month in all_month:
            # get daily dic
            all_day, sub_day_dic = sub_month_dic[month].groupby_mean('day')       

            for day in all_day:
                # get daily dataframe
                sub_day_df = sub_day_dic[day]
                daily_tempo = np.array([year, month, day, sub_day_df.iloc[-1]]).reshape(1,4)
                daily_rain = np.append(daily_rain, daily_tempo, axis = 0)

daily_rain = np.delete(daily_rain, 0, axis = 0)
print(daily_rain.shape)

(6083, 4)


In [20]:
## daily Lychee yield 2004 - 2018
daily_lychee = np.array([1,1,1,1]).reshape(1,4)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')
        # record year month and lychee yield of each month
        daily_lychee_temp = lychee_yield_df.loc[lychee_yield_df['year']==year,:]

        for month in all_month:

                # get daily dic
                all_day, _ = sub_month_dic[month].groupby_mean('day')       

                for day in all_day:    

                    if daily_lychee_temp.size != 0:
                        # print(year, ' : ', month)
                        # print(daily_lychee_temp.iloc[0, month-1])
                        daily_yield = np.array([year, month, day, daily_lychee_temp.iloc[0, month-1]]).reshape(1,4)
                        daily_lychee = np.append(daily_lychee, daily_yield, axis = 0)

daily_lychee = np.delete(daily_lychee, 0, axis = 0)
print(daily_lychee[:6, :])

[[2004    1    1    0]
 [2004    1    2    0]
 [2004    1    3    0]
 [2004    1    4    0]
 [2004    1    5    0]
 [2004    1    6    0]]


In [21]:
## daily Lychee yield 2004 - 2018 yearlyyield
daily_lychee_yearlyyield = np.array([1,1,1,1]).reshape(1,4)
all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')
        # record year month and lychee yield of each month
        daily_lychee_temp = lychee_yield_df.loc[lychee_yield_df['year']==year,:]

        for month in all_month:

                # get daily dic
                all_day, _ = sub_month_dic[month].groupby_mean('day')       

                for day in all_day:    

                    if daily_lychee_temp.size != 0:
                        # print(year, ' : ', month)
                        # print(daily_lychee_temp.iloc[0, month-1])
                        daily_yield = np.array([year, month, day, daily_lychee_temp.iloc[0, -2]]).reshape(1,4)
                        daily_lychee_yearlyyield = np.append(daily_lychee_yearlyyield, daily_yield, axis = 0)

daily_lychee_yearlyyield = np.delete(daily_lychee_yearlyyield, 0, axis = 0)
print(daily_lychee_yearlyyield[:6, :])

[[ 2004     1     1 18399]
 [ 2004     1     2 18399]
 [ 2004     1     3 18399]
 [ 2004     1     4 18399]
 [ 2004     1     5 18399]
 [ 2004     1     6 18399]]


In [22]:
## daily yield per area 2004 - 2018
daily_area = np.array([1,1,1,1]).reshape(1,4)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')
        # record year month and lychee yield of each month
        area_temp = area_df.loc[area_df['year']==year,:]        

        for month in all_month:

                # get daily dic
                all_day, _ = sub_month_dic[month].groupby_mean('day')       

                for day in all_day:    

                    if area_temp.size != 0:
                        daily_area_temp = np.array([year, month, day, area_temp.iloc[0, -1]]).reshape(1,4)
                        daily_area = np.append(daily_area, daily_area_temp, axis = 0)

daily_area = np.delete(daily_area, 0, axis = 0)
print(daily_area.shape)

(5844, 4)


In [23]:
## daily yield per area 2004 - 2018
daily_yieldarea = np.array([1,1,1,1]).reshape(1,4)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')
        # record year month and lychee yield of each month
        area_temp = area_df.loc[area_df['year']==year,:]        

        for month in all_month:

                # get daily dic
                all_day, _ = sub_month_dic[month].groupby_mean('day')       

                for day in all_day:    

                    if area_temp.size != 0:
                        daily_area_temp = np.array([year, month, day, area_temp.iloc[0, 4]]).reshape(1,4)
                        daily_yieldarea = np.append(daily_yieldarea, daily_area_temp, axis = 0)

daily_yieldarea = np.delete(daily_yieldarea, 0, axis = 0)
print(daily_yieldarea.shape)

(5844, 4)


<h1> Rearrange for 3-hourly data </h1>

In [18]:
## hourly Temp 2004 - 2019
threehourly_temp = np.array([1,1,1,1,1]).reshape(1,5)

# get yearly dic
all_year, sub_year_dic = temp_df_clean.groupby_col('year')

for year in all_year:
        # get monthly dic
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')

        for month in all_month:
            # get daily dic
            all_day, sub_day_dic = sub_month_dic[month].groupby_col('day')       

            for day in all_day:
                # get daily dataframe
                sub_day_df = sub_day_dic[day]

                for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                    threehourly_tempo = np.array([year, month, day, int(hour), sub_day_df[hour]]).reshape(1,5)
                    threehourly_temp = np.append(threehourly_temp, threehourly_tempo, axis = 0)

threehourly_temp = np.delete(threehourly_temp, 0, axis = 0)
print(threehourly_temp.shape)

(48664, 5)


In [19]:
## hourly Humid 2004 - 2019
threehourly_humid = np.array([1,1,1,1,1]).reshape(1,5)

# get yearly dic
all_year, sub_year_dic = humid_df_clean.groupby_col('year')

for year in all_year:
        # get monthly dic
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')

        for month in all_month:
            # get daily dic
            all_day, sub_day_dic = sub_month_dic[month].groupby_col('day')       

            for day in all_day:
                # get daily dataframe
                sub_day_df = sub_day_dic[day]

                for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                    threehourly_tempo = np.array([year, month, day, int(hour), sub_day_df[hour]]).reshape(1,5)
                    threehourly_humid = np.append(threehourly_humid, threehourly_tempo, axis = 0)

threehourly_humid = np.delete(threehourly_humid, 0, axis = 0)
print(threehourly_humid.shape)

(48664, 5)


In [33]:
## hourly Rain 2004 - 2019
threehourly_rain = np.array([1,1,1,1,1]).reshape(1,5)

# get yearly dic
all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
        # get monthly dic
        all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')

        for month in all_month:
            # get daily dic
            all_day, sub_day_dic = sub_month_dic[month].groupby_col('day')       

            for day in all_day:
                # get daily dataframe
                sub_day_df = sub_day_dic[day]

                for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                    threehourly_tempo = np.array([year, month, day, int(hour), sub_day_df[hour]]).reshape(1,5)
                    threehourly_rain = np.append(threehourly_rain, threehourly_tempo, axis = 0)

threehourly_rain = np.delete(threehourly_rain, 0, axis = 0)
print(threehourly_rain.shape)

Year 2003 : Month 1
Year 2003 : Month 2
Year 2003 : Month 3
Year 2003 : Month 4
Year 2003 : Month 5
Year 2003 : Month 6
Year 2003 : Month 7
Year 2003 : Month 8
Year 2003 : Month 9
Year 2003 : Month 10
Year 2003 : Month 11
Year 2003 : Month 12
Year 2004 : Month 1
Year 2004 : Month 2
Year 2004 : Month 3
Year 2004 : Month 4
Year 2004 : Month 5
Year 2004 : Month 6
Year 2004 : Month 7
Year 2004 : Month 8
Year 2004 : Month 9
Year 2004 : Month 10
Year 2004 : Month 11
Year 2004 : Month 12
Year 2005 : Month 1
Year 2005 : Month 2
Year 2005 : Month 3
Year 2005 : Month 4
Year 2005 : Month 5
Year 2005 : Month 6
Year 2005 : Month 7
Year 2005 : Month 8
Year 2005 : Month 9
Year 2005 : Month 10
Year 2005 : Month 11
Year 2005 : Month 12
Year 2006 : Month 1
Year 2006 : Month 2
Year 2006 : Month 3
Year 2006 : Month 4
Year 2006 : Month 5
Year 2006 : Month 6
Year 2006 : Month 7
Year 2006 : Month 8
Year 2006 : Month 9
Year 2006 : Month 10
Year 2006 : Month 11
Year 2006 : Month 12
Year 2007 : Month 1
Year 200

In [26]:
## hourly Lychee yield 2004 - 2018
threehourly_lychee = np.array([1,1,1,1,1]).reshape(1,5)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
    all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')
    monthly_lychee_temp = lychee_yield_df.loc[lychee_yield_df['year']==year,:]

    for month in all_month:
        # get daily dic
        all_day, _ = sub_month_dic[month].groupby_col('day')       

        for day in all_day:                
            for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                if monthly_lychee_temp.size != 0:
                    threehourly_yield = np.array([year, month, day, int(hour), monthly_lychee_temp.iloc[0, month-1]]).reshape(1,5)
                    threehourly_lychee = np.append(threehourly_lychee, threehourly_yield, axis = 0)

threehourly_lychee = np.delete(threehourly_lychee, 0, axis = 0)
print(threehourly_lychee.shape)

(43832, 5)


In [ ]:
## hourly Lychee yield 2004 - 2018 yearlyyield
threehourly_lychee_lycheeyield = np.array([1,1,1,1,1]).reshape(1,5)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
    all_month, sub_month_dic = sub_year_dic[year].groupby_col('month')
    monthly_lychee_temp = lychee_yield_df.loc[lychee_yield_df['year']==year,:]

    for month in all_month:
        # get daily dic
        all_day, _ = sub_month_dic[month].groupby_col('day')       

        for day in all_day:                
            for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                if monthly_lychee_temp.size != 0:
                    threehourly_yield = np.array([year, month, day, int(hour), monthly_lychee_temp.iloc[0, month-1]]).reshape(1,5)
                    threehourly_lychee_lycheeyield = np.append(threehourly_lychee_lycheeyield, threehourly_yield, axis = 0)

threehourly_lychee_lycheeyield = np.delete(threehourly_lychee_lycheeyield, 0, axis = 0)
print(threehourly_lychee_lycheeyield[:6,:])

In [38]:
## hourly yield per area 2004 - 2018
threehourly_area = np.array([1,1,1,1,1]).reshape(1,5)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
    all_month, sub_month_dic    = sub_year_dic[year].groupby_col('month')
    monthly_area_temp         = area_df.loc[area_df['year']==year,:]

    for month in all_month:
        # get daily dic
        all_day, _ = sub_month_dic[month].groupby_col('day')       

        for day in all_day:                
            for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                if monthly_area_temp.size != 0:
                    threehourly_area_temp = np.array([year, month, day, int(hour), monthly_area_temp.iloc[0, -1]]).reshape(1,5)
                    threehourly_area = np.append(threehourly_area, threehourly_area_temp, axis = 0)

threehourly_area = np.delete(threehourly_area, 0, axis = 0)
print(threehourly_area.shape)

(46752, 5)


In [39]:
## hourly all area 2004 - 2018
threehourly_yieldarea = np.array([1,1,1,1,1]).reshape(1,5)

all_year, sub_year_dic = rain_df_clean.groupby_col('year')

for year in all_year:
    all_month, sub_month_dic    = sub_year_dic[year].groupby_col('month')
    monthly_area_temp           = area_df.loc[area_df['year']==year,:]

    for month in all_month:
        # get daily dic
        all_day, _ = sub_month_dic[month].groupby_col('day')       

        for day in all_day:                
            for hour in ['1', '4', '7', '10', '13', '16', '19', '22']:
                if monthly_area_temp.size != 0:
                    threehourly_area_temp = np.array([year, month, day, int(hour), monthly_area_temp.iloc[0, 4]]).reshape(1,5)
                    threehourly_yieldarea = np.append(threehourly_yieldarea, threehourly_area_temp, axis = 0)

threehourly_yieldarea = np.delete(threehourly_yieldarea, 0, axis = 0)
print(threehourly_yieldarea.shape)

(46752, 5)


Intersected
 year of all data is 2004 - 2018

<h1> Method for getting X data </h1>

In [24]:
def split_x_sequence(X, n_steps, ind_output):
    
    X_seq = np.ones((1, n_steps))

    for i in range(X.shape[0]):
        
        if i+n_steps > X.shape[0] - 1:
            break
        x_seq_temp = X[i:i+n_steps, ind_output].reshape(1,-1)
        X_seq = np.append(X_seq, x_seq_temp, axis = 0)
    
    return X_seq[1:,:]

def split_y_sequence(X, n_steps, ind_output, repeat):
    
    # repeat = False return only target
    # repeat = true return target with multiple row
    X_seq = np.ones((1, 1)) if repeat == False else np.ones((1, n_steps))

    for i in range(X.shape[0]):
        
        if i+n_steps > X.shape[0] - 1:
            break
        x_seq_temp = X[i+n_steps, ind_output].reshape(1, -1) if repeat == False else  X[i+n_steps, ind_output]*np.ones((1, n_steps))
        X_seq = np.append(X_seq, x_seq_temp, axis = 0)
    
    return X_seq[1:,:]

In [25]:
def get_XY_fr_resolution(df_list_input, df_list_output, n_steps, req_m_input = [1, 2], req_month = [4, 5, 6], axis = 1, yearlyyield = False):

    df_copy_list  = list()
    X             = list()

    # extract X
    count = 0
    for df, xy_split in df_list_input:
        df_copy = df.copy()
        ind_2004_2018 = np.logical_and(df_copy[:, 0] >= 2004, df_copy[:, 0] <= 2018)
        df_copy = df_copy[ind_2004_2018,:]

        # find index only for required input month
        if yearlyyield and count == 0:
            count = count + 1
            req_m_ind = np.zeros((df_copy.shape[0],))
            for i in req_m_input:
                req_m_ind = np.logical_or(req_m_ind, (df_copy[:,1] == i).reshape(-1,))

        df_copy = df_copy[req_m_ind, :]

        if xy_split == 'x':
            X_temp = split_x_sequence(df_copy, n_steps, -1)
            print('true')
            print(X_temp.shape)
        if xy_split == 'y':
            X_temp = split_y_sequence(df_copy, n_steps, -1, True) if axis == 2 else split_y_sequence(df_copy, n_steps, -1, False)
      
        X_temp = X_temp.reshape(X_temp.shape[0], X_temp.shape[1], 1)
        X.append(X_temp)

    # axis = 2 for rnn and axis = 1 for lr and svr
    X = np.concatenate(X, axis = axis)

    # extract y
    Y = split_y_sequence(df_list_output[0], n_steps, -1, False) if yearlyyield == False else split_y_sequence(df_list_output[0][req_m_ind, :], n_steps, -1, False)  

    # get only the data from required month
    all_year    = np.unique(df_copy[:, 0])
    all_month   = np.unique(df_copy[:, 1]) if yearlyyield == False else np.array([])
    req_ind     = np.zeros((X_temp.shape[0], 1))
    for year in all_year:
        for month in req_month:
            year_ind = df_copy[n_steps:, 0] == year
            
            if yearlyyield == False:
                month_ind = df_copy[n_steps:, 1] == month
                ym_ind = year_ind*month_ind
            else:
                ym_ind = year_ind

            ym_ind_where = np.where(ym_ind == 1)
            
            if len(ym_ind_where[0]) != 0:
                if yearlyyield == False:
                    # get only the data from required month only at the first index
                    first_ind = ym_ind_where[0][0]

                else:
                    # get only the data from the last index of each year
                    first_ind = ym_ind_where[0][0]                  

                ym_ind = np.zeros((ym_ind.shape[0], 1))
                
                try:
                    ym_ind[first_ind] = 1
                except:
                    pass

                ym_ind  = ym_ind.reshape(-1, 1)
                req_ind = req_ind + ym_ind

    req_ind = req_ind.astype(bool).reshape(-1)
    X = X[req_ind, :, :]
    Y = Y[req_ind]
    if axis == 1:
        X = X.reshape(X.shape[0], X.shape[1])
        Y = Y.reshape(Y.shape[0])

    n_features = X.shape[2] if axis == 2 else X.shape[1]
    n_size      = X.shape[0]

    year_target = df_copy[n_steps:, 0][req_ind]
    month_target = df_copy[n_steps:, 1][req_ind] if yearlyyield == False else np.array([])

    return X, Y, n_features, n_size, n_steps, year_target, month_target

In [26]:
def rmse(y_true, y_pred):

    ind_ignorezero = (y_true != 0).reshape(-1,)
    error = (y_true - y_pred)
    se = error**2
    mse = np.mean(se)
    rmse = mse**0.5
    return rmse

In [27]:
def mape(y_true, y_pred):
    error = y_true - y_pred
    pe     = (y_true - y_pred)/y_true*100
    ape = np.abs(pe)
    mape = np.mean(ape)

    return mape    

In [28]:
def mae(y_true, y_pred):

    error = y_true - y_pred
    ae = np.abs(error)
    mae = np.mean(ae)

    return mae

In [29]:
def r2(y_true, y_pred):
    y_mean  = np.mean(y_true)
    Stot    = np.sum((y_true - y_mean)**2)
    Sres    = np.sum((y_true - y_pred)**2)
    r_square    = 1 - Sres/Stot

    return r_square

Get y

<h1> Feature selection </h1>
Year (of the target), Month (of the target), Avg.Temp (3 and 4 month before), Avg.Humid (3 and 4 month before),
Rain amount (3 and 4 month before), lychee yield per area (1 year before), number of the day in target month,
lychee yield (of target month but 1 year before)

In [164]:
choose = 'monthly_yearlyyield_wyear'

if choose == 'monthly_yearlyyield_wyear':
    yearlyyield = True
    input_vec = [
        [monthly_temp, 'x'],
        # [monthly_humid, 'x'], 
        # [monthly_rain, 'x'], 
        # [monthly_area, 'x'], 
        # [monthly_yieldarea, 'x'], 
        # [monthly_lychee, 'x'],
        [monthly_yieldarea[:,0].reshape(-1,1), 'y']]   # target year

    output_vec = [monthly_lychee_yearlyyield]

if choose == 'monthly_yearlyyield_woyear':
    yearlyyield = True
    input_vec = [
        [monthly_temp, 'x'],
        # [monthly_humid, 'x'], 
        # [monthly_rain, 'x'], 
        # [monthly_area, 'x'], 
        # [monthly_yieldarea, 'x'], 
        # [monthly_lychee, 'x'],
        ]#[monthly_yieldarea[:,0].reshape(-1,1), 'y']]   # target year

if choose == 'test':
    input_vec = [
        [daily_temp, 'x'], 
        [daily_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [daily_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month

    output_vec = [daily_lychee]

if choose == 'monthly':
    yearlyyield = False
    input_vec = [
        [monthly_temp, 'x'],
        # [monthly_humid, 'x'], 
        # [monthly_rain, 'x'], 
        # [monthly_area, 'x'], 
        [monthly_yieldarea, 'x'], 
        [monthly_lychee, 'x'], 
        [monthly_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [monthly_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month
    
    output_vec = [monthly_lychee]

if choose == 'daily':
    yearlyyield = False
    input_vec = [
        [daily_temp, 'x'],
        [daily_humid, 'x'], 
        # [daily_rain, 'x'], 
        # [daily_area, 'x'], 
        # [daily_yieldarea, 'x'], 
        [daily_lychee, 'x'], 
        [daily_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [daily_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month

    output_vec = [daily_lychee]

if choose == 'hourly':
    yearlyyield = False
    input_vec = [
        [hourly_temp, 'x'],
        # [hourly_humid, 'x'], 
        # [hourly_rain, 'x'], 
        # [hourly_area, 'x'], 
        [hourly_yieldarea, 'x'], 
        [hourly_lychee, 'x'], 
        [hourly_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [hourly_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month

    output_vec = [hourly_lychee]

# daily choose n_steps = 240
# hourly choose n_steps = 1920
# monthly choose n_steps = 4
# axis = 2 for rnn

X, Y, n_features, n_size, n_steps, year_target, month_target = get_XY_fr_resolution(input_vec, output_vec, n_steps = 2, axis = 1, req_m_input = [1, 2], req_month = [4, 5, 6], yearlyyield = yearlyyield)

true
(28, 2)
true
(28, 2)


In [165]:
print('X Shape')
print(X.shape)
print('Y Shape')
print(Y.shape)
print('n features')
print(n_features)
print('sample size')
print(n_size)

X Shape
(14, 5)
Y Shape
(14,)
n features
5
sample size
14


In [166]:
print(X)

[[1.99548387e+01 2.15482759e+01 4.14370000e+04 4.14370000e+04
  2.00500000e+03]
 [2.03677419e+01 2.27607143e+01 4.33660000e+04 4.33660000e+04
  2.00600000e+03]
 [2.00032258e+01 2.26857143e+01 4.71910000e+04 4.71910000e+04
  2.00700000e+03]
 [1.98451613e+01 2.10250000e+01 4.89860000e+04 4.89860000e+04
  2.00800000e+03]
 [2.02290323e+01 2.17379310e+01 4.37850000e+04 4.37850000e+04
  2.00900000e+03]
 [1.95387097e+01 2.28107143e+01 3.42520000e+04 3.42520000e+04
  2.01000000e+03]
 [2.19516129e+01 2.23821429e+01 3.22190000e+04 3.22190000e+04
  2.01100000e+03]
 [2.07709677e+01 2.20071429e+01 3.24620000e+04 3.24620000e+04
  2.01200000e+03]
 [2.05806452e+01 2.27620690e+01 3.21100000e+04 3.21100000e+04
  2.01300000e+03]
 [2.11129032e+01 2.41250000e+01 3.04500000e+04 3.04500000e+04
  2.01400000e+03]
 [1.92741935e+01 2.20142857e+01 2.30010000e+04 2.30010000e+04
  2.01500000e+03]
 [1.96387097e+01 2.17357143e+01 2.04130000e+04 2.04130000e+04
  2.01600000e+03]
 [1.89709677e+01 2.15931034e+01 1.940000

Seperate train set and test set for displaying

In [167]:
X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size=0.3)
X_train_ord = X[:int(n_size*(1-0.3)), :]
X_test_ord  = X[int(n_size*(1-0.3)):, :]
Y_train_ord = Y[:int(n_size*(1-0.3))]
Y_test_ord  = Y[int(n_size*(1-0.3)):]
print('train size')
print(X_train.shape)
print('test size')
print(X_test.shape)

train size
(9, 5)
test size
(5, 5)


<h1> Ridge Model w/o Normalization </h1>

In [168]:
# vector for record error
train_mae = np.array([])
test_mae = np.array([])

train_rmse = np.array([])
test_rmse = np.array([])

train_r2 = np.array([])
test_r2 = np.array([])

# using grid searchcv to optimize hyper param
reg_lr  = Ridge()
param   = {'alpha':[0.01, 0.001, 0.1, 0.5, 1, 5, 10, 100, 200, 300, 400, 1000]}
gsc     = GridSearchCV(reg_lr, param, cv = 5)
gsc.fit(X, np.log1p(Y))

reg_lr  = gsc.best_estimator_
print(gsc.best_estimator_)

Ridge(alpha=1000, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [169]:
reg_lr.fit(X_train_ord, np.log1p(Y_train_ord))

Y_all_test = reg_lr.predict(X)
Y_all_test[Y_all_test<0] = 0

Y_all_test = np.expm1(Y_all_test)

train_mae = mae(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')

if yearlyyield == False:
    for year, month, y_true, y_pred in zip(year_target, month_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))
else:
    for year, y_true, y_pred in zip(year_target, Y, Y_all_test):
        print('year {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, y_true, y_pred))   

train MAE 2541.08	test MAE 661.21
train RMSE 3022.76	test RMSE 880.94
train MAPE 18.34	test MAPE 10.87
year 2005 true : 17938	pred : 14646
year 2006 true : 19773	pred : 15950
year 2007 true : 19986	pred : 18909
year 2008 true : 14055	pred : 20459
year 2009 true : 18252	pred : 16174
year 2010 true : 9251	pred : 10526
year 2011 true : 8187	pred : 9591
year 2012 true : 11945	pred : 9681
year 2013 true : 8263	pred : 9515
year 2014 true : 7084	pred : 8820
year 2015 true : 5899	pred : 6300
year 2016 true : 4832	pred : 5599
year 2017 true : 5408	pred : 5342
year 2018 true : 5650	pred : 5314


In [170]:
# export output
# X_rnn[:int(train_ratio*n_size), :, :]
year_train  = year_target[:int(n_size*(1-0.3))]
month_train = month_target[:int(n_size*(1-0.3))] if yearlyyield == False else year_train
year_test   = year_target[int(n_size*(1-0.3)):]
month_test  = month_target[int(n_size*(1-0.3)):] if yearlyyield == False else year_test
y_train_true = Y[:int(n_size*(1-0.3))]
y_train_pred = Y_all_test[:int(n_size*(1-0.3))]
y_test_true = Y[int(n_size*(1-0.3)):]
y_test_pred = Y_all_test[int(n_size*(1-0.3)):]
export_output('ridge_results_monthly_input_yearly_output_v3.xlsx', year_train, month_train, y_train_true, y_train_pred, year_test, month_test, y_test_true, y_test_pred)

<h1> Ridge Model w Normalization </h1>

In [171]:
# vector for record error
train_mae = np.array([])
test_mae = np.array([])

train_rmse = np.array([])
test_rmse = np.array([])

train_r2 = np.array([])
test_r2 = np.array([])

# using grid searchcv to optimize hyper param
estimators = [('normalize', MinMaxScaler()), ('lr', Ridge())]
pipe    = Pipeline(estimators)
param   = dict(lr__alpha=[0.01, 0.001, 0.1, 0.5, 1, 5, 10, 100])

gsc     = GridSearchCV(pipe, param, cv = 5)
gsc.fit(X, np.log1p(Y))

reg_lr_norm  = gsc.best_estimator_
print(gsc.best_estimator_)

Pipeline(memory=None,
     steps=[('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))), ('lr', Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [172]:
reg_lr_norm.fit(X_train_ord, np.log1p(Y_train_ord))

Y_all_test = reg_lr_norm.predict(X)
Y_all_test[Y_all_test<0] = 0
Y_all_test = np.expm1(Y_all_test)

train_mae = mae(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')

if yearlyyield == False:
    for year, month, y_true, y_pred in zip(year_target, month_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))
else:
    for year, y_true, y_pred in zip(year_target, Y, Y_all_test):
        print('year {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, y_true, y_pred))   

train MAE 2297.95	test MAE 853.32
train RMSE 2454.53	test RMSE 985.37
train MAPE 17.08	test MAPE 14.63
year 2005 true : 17938	pred : 16445
year 2006 true : 19773	pred : 16788
year 2007 true : 19986	pred : 18038
year 2008 true : 14055	pred : 17971
year 2009 true : 18252	pred : 14992
year 2010 true : 9251	pred : 11319
year 2011 true : 8187	pred : 9877
year 2012 true : 11945	pred : 9702
year 2013 true : 8263	pred : 9341
year 2014 true : 7084	pred : 8633
year 2015 true : 5899	pred : 6801
year 2016 true : 4832	pred : 6039
year 2017 true : 5408	pred : 5698
year 2018 true : 5650	pred : 5332


<h1> Support Vector Regression w/o Normalization </h1>

In [173]:
# vector for record error
train_mae = np.array([])
test_mae = np.array([])

train_rmse = np.array([])
test_rmse = np.array([])

train_r2 = np.array([])
test_r2 = np.array([])

# using grid searchcv to optimize hyper param
estimators = [('svr', SVR(kernel = 'linear', gamma = 'scale'))]
pipe    = Pipeline(estimators)
param   = dict(
    svr__C=[0.001, 0.01, 0.1, 0.5, 1, 5, 100],
    svr__epsilon = [0.001, 0.01, 0.1, 0.5, 1])

gsc     = GridSearchCV(pipe, param, cv = 3)
gsc.fit(X, np.log1p(Y))
# gsc.fit(X, Y)

reg_svr  = gsc.best_estimator_
print(gsc.best_estimator_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
Pipeline(memory=None,
     steps=[('svr', SVR(C=0.001, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])


In [174]:
reg_svr.fit(X_train_ord, np.log1p(Y_train_ord))
# reg_svr.fit(X_train_ord, Y_train_ord)

Y_all_test = reg_svr.predict(X)
Y_all_test[Y_all_test<0] = 0
Y_all_test = np.expm1(Y_all_test)

train_mae = mae(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')

if yearlyyield == False:
    for year, month, y_true, y_pred in zip(year_target, month_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))
else:
    for year, y_true, y_pred in zip(year_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))   

train MAE 2467.46	test MAE 662.71
train RMSE 3118.11	test RMSE 773.77
train MAPE 16.59	test MAPE 11.22
year 2005 month 8 true : 17938	pred : 14458
year 2006 month 8 true : 19773	pred : 15761
year 2007 month 8 true : 19986	pred : 18887
year 2008 month 8 true : 14055	pred : 20514
year 2009 month 8 true : 18252	pred : 15784
year 2010 month 8 true : 9251	pred : 9834
year 2011 month 8 true : 8187	pred : 8824
year 2012 month 8 true : 11945	pred : 8884
year 2013 month 8 true : 8263	pred : 8671
year 2014 month 8 true : 7084	pred : 7928
year 2015 month 8 true : 5899	pred : 5490
year 2016 month 8 true : 4832	pred : 4804
year 2017 month 8 true : 5408	pred : 4545
year 2018 month 8 true : 5650	pred : 4480


In [177]:
# export output
# X_rnn[:int(train_ratio*n_size), :, :]
year_train  = year_target[:int(n_size*(1-0.3))]
month_train = month_target[:int(n_size*(1-0.3))] if yearlyyield == False else year_train
year_test   = year_target[int(n_size*(1-0.3)):]
month_test  = month_target[int(n_size*(1-0.3)):] if yearlyyield == False else year_test
y_train_true = Y[:int(n_size*(1-0.3))]
y_train_pred = Y_all_test[:int(n_size*(1-0.3))]
y_test_true = Y[int(n_size*(1-0.3)):]
y_test_pred = Y_all_test[int(n_size*(1-0.3)):]
export_output('svr_results_monthly_input_yearly_output_v3.xlsx', year_train, month_train, y_train_true, y_train_pred, year_test, month_test, y_test_true, y_test_pred)

<h1> Support Vector Regression w Normalization </h1>

In [175]:
# vector for record error
train_mae = np.array([])
test_mae = np.array([])

train_rmse = np.array([])
test_rmse = np.array([])

train_r2 = np.array([])
test_r2 = np.array([])

# using grid searchcv to optimize hyper param
estimators = [('normalize', MinMaxScaler()),('svr', SVR(kernel = 'linear', gamma = 'scale'))]
pipe    = Pipeline(estimators)
param   = dict(
    svr__C=[0.001, 0.01, 0.1, 0.5, 1, 5, 100],
    svr__epsilon = [0.001, 0.01, 0.1, 0.5, 1])

gsc     = GridSearchCV(pipe, param, cv = 3)
gsc.fit(X, np.log1p(Y))

reg_svr_norm  = gsc.best_estimator_
print(gsc.best_estimator_)

Pipeline(memory=None,
     steps=[('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))), ('svr', SVR(C=0.5, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False))])
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [176]:
reg_svr_norm.fit(X_train_ord, np.log1p(Y_train_ord))

Y_all_test = reg_svr_norm.predict(X)
Y_all_test[Y_all_test<0] = 0
Y_all_test = np.expm1(Y_all_test)

train_mae = mae(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\t\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')

if yearlyyield == False:
    for year, month, y_true, y_pred in zip(year_target, month_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))
else:
    for year, y_true, y_pred in zip(year_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))   

train MAE 2307.98	test MAE 707.62
train RMSE 2513.20	test RMSE 817.80
train MAPE 16.74		test MAPE 12.09
year 2005 month 8 true : 17938	pred : 16223
year 2006 month 8 true : 19773	pred : 16605
year 2007 month 8 true : 19986	pred : 18089
year 2008 month 8 true : 14055	pred : 18283
year 2009 month 8 true : 18252	pred : 15031
year 2010 month 8 true : 9251	pred : 11053
year 2011 month 8 true : 8187	pred : 9608
year 2012 month 8 true : 11945	pred : 9497
year 2013 month 8 true : 8263	pred : 9134
year 2014 month 8 true : 7084	pred : 8376
year 2015 month 8 true : 5899	pred : 6555
year 2016 month 8 true : 4832	pred : 5800
year 2017 month 8 true : 5408	pred : 5480
year 2018 month 8 true : 5650	pred : 5099


<h1>Get X and y for Recurrent Neural Network</h1>

method for getting x data for rnn

In [184]:
choose = 'monthly_yearlyyield_wyear'

if choose == 'monthly_yearlyyield_wyear':
    yearlyyield = True
    input_vec = [
        [monthly_temp, 'x'],
        # [monthly_humid, 'x'], 
        # [monthly_rain, 'x'], 
        # [monthly_area, 'x'], 
        # [monthly_yieldarea, 'x'], 
        # [monthly_lychee, 'x'],
        [monthly_yieldarea[:,0].reshape(-1,1), 'y']]   # target year

    output_vec = [monthly_lychee_yearlyyield]

if choose == 'monthly_yearlyyield_woyear':
    yearlyyield = True
    input_vec = [
        [monthly_temp, 'x'],
        # [monthly_humid, 'x'], 
        # [monthly_rain, 'x'], 
        # [monthly_area, 'x'], 
        # [monthly_yieldarea, 'x'], 
        # [monthly_lychee, 'x'],
        ]#[monthly_yieldarea[:,0].reshape(-1,1), 'y']]   # target year

if choose == 'test':
    input_vec = [
        [daily_temp, 'x'], 
        [daily_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [daily_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month

    output_vec = [daily_lychee]

if choose == 'monthly':
    yearlyyield = False
    input_vec = [
        [monthly_temp, 'x'],
        # [monthly_humid, 'x'], 
        # [monthly_rain, 'x'], 
        # [monthly_area, 'x'], 
        [monthly_yieldarea, 'x'], 
        [monthly_lychee, 'x'], 
        [monthly_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [monthly_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month
    
    output_vec = [monthly_lychee]

if choose == 'daily':
    yearlyyield = False
    input_vec = [
        [daily_temp, 'x'],
        [daily_humid, 'x'], 
        # [daily_rain, 'x'], 
        # [daily_area, 'x'], 
        # [daily_yieldarea, 'x'], 
        [daily_lychee, 'x'], 
        [daily_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [daily_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month

    output_vec = [daily_lychee]

if choose == 'hourly':
    yearlyyield = False
    input_vec = [
        [hourly_temp, 'x'],
        # [hourly_humid, 'x'], 
        # [hourly_rain, 'x'], 
        # [hourly_area, 'x'], 
        [hourly_yieldarea, 'x'], 
        [hourly_lychee, 'x'], 
        [hourly_yieldarea[:,0].reshape(-1,1), 'y'],   # target year
        [hourly_yieldarea[:,0:2].reshape(-1,2), 'y']] # target month

    output_vec = [hourly_lychee]

# daily choose n_steps = 240
# hourly choose n_steps = 1920
# monthly choose n_steps = 4
# axis = 2 for rnn

########## choose architecture of neural network

choose = 'gru'

X_rnn, Y_rnn, n_features, n_size, n_steps, year_target, month_target = get_XY_fr_resolution(input_vec, output_vec, n_steps = 2, axis = 1 if choose == 'ann' else 2, req_m_input = [1, 2], req_month = [4, 5, 6], yearlyyield = yearlyyield)

print('n_steps : {:d} n_features : {:d} n_size : {:d}'.format(n_steps, n_features, n_size))

true
(28, 2)
true
(28, 2)
n_steps : 2 n_features : 3 n_size : 14


Seperate train set and test set with unshuffle for displaying</br>
test size = 53
train size = 124

In [186]:
train_ratio = 0.7

X_train_ord         = X_rnn[:int(train_ratio*n_size), :] if choose == 'ann' else X_rnn[:int(train_ratio*n_size), :, :]
X_test_ord          = X_rnn[int(train_ratio*n_size):, :] if choose == 'ann' else X_rnn[int(train_ratio*n_size):, :, :]
Y_train_ord         = Y_rnn[:int(train_ratio*n_size)].reshape(-1)
Y_test_ord          = Y_rnn[int(train_ratio*n_size):].reshape(-1)
print(Y_train_ord.shape)

(9,)


In [187]:
X_train     = X_train_ord
X_test      = X_test_ord
Y_train     = Y_train_ord
Y_test      = Y_test_ord

print(X_train.shape)
print(Y_train.shape)

(9, 2, 3)
(9,)


<h1> Recurrent Neural Network w/o Normalization </h1>

In [188]:
class Scaler3D():

    def __init__(self):
        self.scaler_list = {}

    def fit(self, x):
        self.x = x
        
        min_list = np.array([])
        max_list = np.array([])

        len_feature = x.shape[-1]

        for i in range(len_feature):
            self.scaler_list[i] = MinMaxScaler()
            self.scaler_list[i].fit(x[:, :, i])

    def transform(self, x):
        x_copy = x.copy()

        len_feature = x.shape[-1]

        for i in range(len_feature):
            x_copy[:,:,i] = self.scaler_list[i].transform(x[:, :, i])

        return x_copy

In [189]:
def create_model_lstm(n_steps, n_features):
    rnn = Sequential()
    rnn.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(50, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(50, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(1, activation='relu'))
    rnn.compile(optimizer='adam', loss='mse')

    return rnn  

In [190]:
def create_model_gru(n_steps, n_features):
    
    rnn = Sequential()
    rnn.add(GRU(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    rnn.add(Dropout(0.4))
    rnn.add(GRU(100, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.4))
    rnn.add(GRU(100, activation='relu'))    
    # rnn.add(BatchNormalization())    
    rnn.add(Dropout(0.4))
    rnn.add(Dense(50, activation='relu'))
    rnn.add(Dropout(0.4))    
    rnn.add(Dense(1, activation='relu'))
    rnn.compile(optimizer='adam', loss='mse')
    return rnn  

In [191]:
def create_model_ann(n_steps, n_features):
    reg = l1(0.01)
    rnn = Sequential()
    # input layer
    rnn.add(Dense(128, activation='relu', input_dim=n_features))
    # hidden layer
    rnn.add(Dense(256, activation='relu',
                kernel_regularizer=reg)) 
    rnn.add(Dropout(0.4))
    rnn.add(Dense(256, activation='relu',
                kernel_regularizer=reg))   
    rnn.add(Dropout(0.4)) 
    rnn.add(Dense(256, activation='relu',
                kernel_regularizer=reg))   
    rnn.add(Dropout(0.4))     
    # output layer
    rnn.add(Dense(1, activation='relu'))
    rnn.compile(optimizer='adam', loss='mse')
    return rnn  

<h1> Recurrent Neural Network w Normalization </h1>

In [200]:
# Create a callback that saves the model's weights
if choose == 'lstm':
    checkpoint_path = './checkpoint_path_lstm'
    rnn = create_model_lstm(n_steps, n_features)   
elif choose == 'gru':
    checkpoint_path = './checkpoint_path_gru_3layer_monthly_yearlyyield.ckpt'
    rnn = create_model_gru(n_steps, n_features)   
else:
    checkpoint_path = './checkpoint_path_ann_3layer_monthly_yearlyyield.ckpt'
    rnn = create_model_ann(n_steps, n_features)       

cp_callback     = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0)        
rnn.summary()        

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_9 (GRU)                  (None, 2, 100)            31500     
_________________________________________________________________
dropout_12 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
gru_10 (GRU)                 (None, 2, 100)            60600     
_________________________________________________________________
dropout_13 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
gru_11 (GRU)                 (None, 100)               60600     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)               

In [205]:
# vector for record error
train_mae = np.array([])
test_mae = np.array([])

train_rmse = np.array([])
test_rmse = np.array([])

train_r2 = np.array([])
test_r2 = np.array([])

# minmaxscaler
scaler = Scaler3D()
scaler.fit(X_train)

for i in range(300):

    # if os.path.isfile(checkpoint_path):
    #     rnn.load_weights(checkpoint_path)

        if choose != 'ann':
            rnn.fit(scaler.transform(X_train), np.log1p(Y_train), epochs=50, verbose=0, batch_size=32)#, callbacks=[cp_callback])

            Y_test_pred = rnn.predict(scaler.transform(X_test))
            Y_test_pred[Y_test_pred<0] = 0

            Y_train_pred = rnn.predict(scaler.transform(X_train))
            Y_train_pred[Y_train_pred<0] = 0

            Y_test_pred = np.expm1(Y_test_pred)
            Y_train_pred = np.expm1(Y_train_pred)

        else:
            rnn.fit(scaler.transform(X_train), np.log1p(Y_train), epochs=5, verbose=1, batch_size=32)#, callbacks=[cp_callback])

            Y_test_pred = rnn.predict(X_test)
            Y_test_pred[Y_test_pred<0] = 0

            Y_train_pred = rnn.predict(X_train)
            Y_train_pred[Y_train_pred<0] = 0

            Y_test_pred = np.expm1(Y_test_pred)
            Y_train_pred = np.expm1(Y_train_pred)

            Y_train = np.expm1(Y_train)
            Y_test  = np.expm1(Y_test)

        train_mae = np.append(train_mae, mae(Y_train, Y_train_pred))
        test_mae = np.append(test_mae, mae(Y_test, Y_test_pred))

        train_rmse = np.append(train_rmse, rmse(Y_train, Y_train_pred))
        test_rmse = np.append(test_rmse, rmse(Y_test, Y_test_pred)) 

        if i%10 == 0:
            print('---- Iter : {:d} ----'.format(i))

            print('Train set MAE {:.2f}'.format(np.mean(train_mae[-1])))
            print('Test set MAE {:.2f}'.format(np.mean(test_mae[-1])))

            print('Train set RMSE {:.2f}'.format(np.mean(train_rmse[-1])))
            print('Test set RMSE {:.2f}'.format(np.mean(test_rmse[-1])))  

---- Iter : 0 ----
Train set MAE 4983.34
Test set MAE 2605.03
Train set RMSE 6149.65
Test set RMSE 2711.06
---- Iter : 10 ----
Train set MAE 5084.22
Test set MAE 2558.06
Train set RMSE 6340.46
Test set RMSE 2670.75
---- Iter : 20 ----
Train set MAE 4970.00
Test set MAE 2671.87
Train set RMSE 6147.04
Test set RMSE 2776.75
---- Iter : 30 ----
Train set MAE 5051.69
Test set MAE 2532.39
Train set RMSE 6215.97
Test set RMSE 2645.32
---- Iter : 40 ----
Train set MAE 5013.21
Test set MAE 2882.98
Train set RMSE 6224.93
Test set RMSE 2987.43
---- Iter : 50 ----
Train set MAE 5085.69
Test set MAE 3297.35
Train set RMSE 6397.43
Test set RMSE 3389.54
---- Iter : 60 ----
Train set MAE 5054.62
Test set MAE 3135.19
Train set RMSE 6292.34
Test set RMSE 3232.71
---- Iter : 70 ----
Train set MAE 5089.15
Test set MAE 3297.77
Train set RMSE 6403.02
Test set RMSE 3388.83
---- Iter : 80 ----
Train set MAE 5031.66
Test set MAE 2933.66
Train set RMSE 6263.89
Test set RMSE 3032.24
---- Iter : 90 ----
Train set

In [206]:
Y_all_test = rnn.predict(scaler.transform(X_rnn))
Y_all_test[Y_all_test<0] = 0

Y_all_test = np.expm1(Y_all_test)
# Y_all_test = scaler_Y.inverse_transform(Y_all_test)

train_mae = mae(Y_rnn[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y_rnn[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y_rnn[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y_rnn[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\t\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')

if yearlyyield == False:
    for year, month, y_true, y_pred in zip(year_target, month_target, Y, Y_all_test):
        print('year {:.0f} month {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, month, y_true, y_pred))
else:
    for year, y_true, y_pred in zip(year_target, Y, Y_all_test):
        print('year {:.0f} true : {:.0f}\tpred : {:.0f}'.format(year, y_true, y_pred[0]))   

train MAE 141.59	test MAE 2680.41
train RMSE 157.86	test RMSE 2781.53
train MAPE 42.40		test MAPE 48.73
year 2005 true : 17938	pred : 18134
year 2006 true : 19773	pred : 19869
year 2007 true : 19986	pred : 19861
year 2008 true : 14055	pred : 14204
year 2009 true : 18252	pred : 18323
year 2010 true : 9251	pred : 9220
year 2011 true : 8187	pred : 8458
year 2012 true : 11945	pred : 12072
year 2013 true : 8263	pred : 8471
year 2014 true : 7084	pred : 8455
year 2015 true : 5899	pred : 8463
year 2016 true : 4832	pred : 8451
year 2017 true : 5408	pred : 8461
year 2018 true : 5650	pred : 8445


In [252]:
# export output
# X_rnn[:int(train_ratio*n_size), :, :]
year_train  = year_target[:int(n_size*(1-0.3))]
month_train = month_target[:int(n_size*(1-0.3))] if yearlyyield == False else year_train
year_test   = year_target[int(n_size*(1-0.3)):]
month_test  = month_target[int(n_size*(1-0.3)):] if yearlyyield == False else year_test
y_train_true = Y[:int(n_size*(1-0.3))]
y_train_pred = Y_all_test[:int(n_size*(1-0.3))]
y_test_true = Y[int(n_size*(1-0.3)):]
y_test_pred = Y_all_test[int(n_size*(1-0.3)):]
export_output('rnn_results_monthly_input_yearly_output.xlsx', year_train, month_train, y_train_true, y_train_pred, year_test, month_test, y_test_true, y_test_pred)